Q2.1
**1. Performance (GMWAX).** Compute mean, volatility, and Sharpe ratio for GMWAX over three samples:
* inception - 2011
* 2012 - present
* inception - present
Has the mean, vol, and Sharpe changed much since the case?

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

if 'part2_data_ready' not in globals():
    file_path = Path('data/gmo_analysis_data.xlsx')
    total_returns = pd.read_excel(file_path, sheet_name='total returns', parse_dates=['date']).sort_values('date')
    risk_free = pd.read_excel(file_path, sheet_name='risk-free rate', parse_dates=['date'])
    risk_free = risk_free.rename(columns={'TBill 3M': 'rf_annual'})
    risk_free['rf_monthly'] = risk_free['rf_annual'] / 12.0
    part2_data = total_returns.merge(risk_free[['date', 'rf_monthly']], on='date', how='left').set_index('date')
    for column in ['SPY', 'GMWAX', 'GMGEX']:
        part2_data[f'{column}_excess'] = part2_data[column] - part2_data['rf_monthly']
    years = part2_data.index.to_series().dt.year
    part2_samples = {
        'Inception to 2011': part2_data.loc[years.le(2011)],
        '2012 to present': part2_data.loc[years.ge(2012)],
        'Inception to present': part2_data
    }
    pd.options.display.float_format = lambda value: f'{value:0.4f}'
    part2_data_ready = True

def annualized_stats(series):
    mean_month = series.mean()
    vol_month = series.std(ddof=1)
    mean_ann = mean_month * 12
    vol_ann = vol_month * np.sqrt(12)
    sharpe = mean_ann / vol_ann if vol_ann != 0 else float('nan')
    return mean_ann, vol_ann, sharpe

def max_drawdown(total_series):
    cumulative = (1 + total_series).cumprod()
    running_peak = cumulative.cummax()
    drawdown = cumulative / running_peak - 1
    return drawdown.min()

def summarize_performance(asset):
    rows = []
    for label, frame in part2_samples.items():
        mean_ann, vol_ann, sharpe = annualized_stats(frame[f'{asset}_excess'])
        rows.append({'Sample': label, 'Mean': mean_ann, 'Volatility': vol_ann, 'Sharpe': sharpe})
    return pd.DataFrame(rows)

gmwax_perf = summarize_performance('GMWAX').round(4)
print(gmwax_perf.to_string(index=False))

              Sample   Mean  Volatility  Sharpe
   Inception to 2011 0.0464      0.1105  0.4201
     2012 to present 0.0492      0.0927  0.5305
Inception to present 0.0477      0.1022  0.4670


Mean return holds close to zero point zero five annualized across the subsamples, volatility eased from about zero point eleven to zero point zero nine after two thousand eleven, and the Sharpe ratio only rose from roughly zero point forty to the low zero point fifties. The inception to present statistics fall between those endpoints, so the fund behaves almost the same as it did in the original case.

Q2.2
**2. Tail risk (GMWAX).** For all three samples, analyze extreme scenarios:
* minimum return
* 5th percentile (VaR-5th)
* maximum drawdown (compute on total returns, not excess returns)
(a) Does GMWAX have high or low tail-risk as seen by these stats?
(b) Does that vary much across the two subsamples?

In [2]:
def summarize_tail(asset):
    rows = []
    for label, frame in part2_samples.items():
        excess = frame[f'{asset}_excess']
        total = frame[asset]
        rows.append({
            'Sample': label,
            'Minimum': excess.min(),
            'VaR_0.05': excess.quantile(0.05),
            'Max_Drawdown': max_drawdown(total)
        })
    return pd.DataFrame(rows)

gmwax_tail = summarize_tail('GMWAX').round(4)
print(gmwax_tail.to_string(index=False))

              Sample  Minimum  VaR_0.05  Max_Drawdown
   Inception to 2011  -0.1492   -0.0440       -0.2936
     2012 to present  -0.1150   -0.0398       -0.2168
Inception to present  -0.1492   -0.0411       -0.2936


(a) Monthly losses rarely exceed roughly minus zero point one five and the one in twenty loss sits near minus zero point zero four, while peak to trough declines stayed just under minus zero point three, so the strategy carries only moderate tail-risk relative to equities. (b) The post two thousand eleven subsample shows slightly smaller drawdowns and higher left tail outcomes, but the change is modest enough that tail-risk looks stable over time.

Q2.3
**3. Market exposure (GMWAX).** For all three samples, regress excess returns of GMWAX on excess returns of SPY:
* report estimated alpha, beta, and $R^{2}$
* is GMWAX a low-beta strategy? has that changed since the case?
* does GMWAX provide alpha? has that changed across subsamples?

In [3]:
def summarize_regression(asset):
    rows = []
    for label, frame in part2_samples.items():
        y = frame[f'{asset}_excess'].to_numpy()
        x = frame['SPY_excess'].to_numpy()
        mask = np.isfinite(y) & np.isfinite(x)
        y_valid = y[mask]
        x_valid = x[mask]
        design = np.column_stack([np.ones(len(x_valid)), x_valid])
        coeffs, _, _, _ = np.linalg.lstsq(design, y_valid, rcond=None)
        fitted = design @ coeffs
        resid = y_valid - fitted
        ss_res = np.sum(resid ** 2)
        ss_tot = np.sum((y_valid - y_valid.mean()) ** 2)
        r_squared = 1 - ss_res / ss_tot if ss_tot != 0 else float('nan')
        rows.append({'Sample': label, 'Alpha': coeffs[0], 'Beta': coeffs[1], 'R2': r_squared})
    return pd.DataFrame(rows)

gmwax_reg = summarize_regression('GMWAX').round(4)
print(gmwax_reg.to_string(index=False))

              Sample   Alpha   Beta     R2
   Inception to 2011  0.0023 0.5421 0.6487
     2012 to present -0.0023 0.5669 0.7309
Inception to present  0.0002 0.5475 0.6752


Betas cluster near zero point fifty five in every sample, so GMWAX remains a low beta fund. The pre two thousand twelve alpha is mildly positive, the recent subsample turns slightly negative, and the full period estimate is effectively zero, implying that any alpha highlighted in the case has faded. R squared stays near two thirds, so market exposure continues to explain most variation.

Q2.4
**4. Compare to GMGEX.** Repeat items 1-3 for GMGEX. What are key differences between the two strategies?

In [6]:
gmgex_perf = summarize_performance('GMGEX').round(4)
gmgex_tail = summarize_tail('GMGEX').round(4)
gmgex_reg = summarize_regression('GMGEX').round(4)

print('Performance for GMGEX')
print(gmgex_perf.to_string(index=False))
print('Tail risk for GMGEX')
print(gmgex_tail.to_string(index=False))
print('Regression on SPY excess returns')
print(gmgex_reg.to_string(index=False))

Performance for GMGEX
              Sample    Mean  Volatility  Sharpe
   Inception to 2011 -0.0038      0.1473 -0.0260
     2012 to present  0.0132      0.2281  0.0578
Inception to present  0.0043      0.1900  0.0227
Tail risk for GMGEX
              Sample  Minimum  VaR_0.05  Max_Drawdown
   Inception to 2011  -0.1516   -0.0823       -0.5556
     2012 to present  -0.6589   -0.0656       -0.7374
Inception to present  -0.6589   -0.0757       -0.7618
Regression on SPY excess returns
              Sample   Alpha   Beta     R2
   Inception to 2011 -0.0026 0.7642 0.7259
     2012 to present -0.0081 0.8213 0.2532
Inception to present -0.0051 0.7816 0.3984


GMGEX delivers much lower annualized mean returns, far higher volatility, and significantly deeper drawdowns (over minus zero point five in the early sample and minus zero point seven during the fund crisis window). Its beta sits around zero point eight in both subsamples, so it carries materially more market exposure than GMWAX. Alpha estimates are negative across the board, underscoring that GMGEX has not been paid for the extra risk relative to the steadier GMWAX profile.